<a href="https://colab.research.google.com/github/Prajwalng2/IR-Project/blob/main/IR_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install langdetect transformers scikit-learn nltk


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 10.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993222 sha256=592234b2b4a3a7c237ba05f1e310696eb724ebdc30968578085950bdd9544449
  Stored in directory: /root/.cache/pip/wheels/95/03/7d/59ea870c70ce4e5a370638b5462a7711ab78fba2f655d05106
Successfully built langdetect


In [ ]:
!pip install numpy
!pip install pandas
!pip install scikit-learn
!pip install nltk
!pip install langdetect
!pip install transformers
!pip install torch
!pip install rank-bm25


In [ ]:
import numpy as np
from typing import List, Dict, Union, Optional
from dataclasses import dataclass
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from langdetect import detect
from transformers import MarianMTModel, MarianTokenizer
import torch
from rank_bm25 import BM25Okapi

In [ ]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [ ]:
!pip install transformers torch nltk langdetect sklearn datetime

  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


In [ ]:
import numpy as np
from typing import List, Dict, Union, Optional, Set
from dataclasses import dataclass
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from langdetect import detect
from transformers import MarianMTModel, MarianTokenizer, pipeline
import logging
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import datetime
from collections import defaultdict

# Set up logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Download NLTK data
try:
    nltk.download('punkt', quiet=True)
    nltk.download('stopwords', quiet=True)
except Exception as e:
    logger.warning(f"NLTK download warning: {e}")

@dataclass
class Document:
    id: str
    text: str
    language: str
    metadata: Dict = None
    timestamp: datetime.datetime = None
    category: str = None
    keywords: Set[str] = None

class EnhancedCrossLanguageRetriever:
    def __init__(self):
        """Initialize the enhanced cross-language retrieval system"""
        # Extended language support
        self.language_pairs = {
            'en': 'English',
            'es': 'Spanish',
            'fr': 'French',
            'de': 'German',
            'it': 'Italian',
            'pt': 'Portuguese',
            'nl': 'Dutch',
            'pl': 'Polish',
            'ru': 'Russian',
            'zh': 'Chinese',
            'ja': 'Japanese',
            'ko': 'Korean',
            'ar': 'Arabic',
            'hi': 'Hindi'
        }

        # Initialize translation models and sentiment analyzer
        self.translation_models = {}
        self.tokenizers = {}
        self.sentiment_analyzer = pipeline('sentiment-analysis')
        self.initialize_translation_models()

        # Enhanced document storage with indexing
        self.documents = {}
        self.vectorizers = {}
        self.language_indexes = defaultdict(list)
        self.category_indexes = defaultdict(list)
        self.keyword_indexes = defaultdict(list)
        self.date_index = defaultdict(list)

        # Cache for translation results
        self.translation_cache = {}

    def initialize_translation_models(self):
        """Initialize translation models for language pairs"""
        logger.info("Initializing translation models...")
        primary_languages = ['en', 'es', 'fr', 'de', 'it']  # Core languages

        for source_lang in primary_languages:
            for target_lang in primary_languages:
                if source_lang != target_lang:
                    model_name = f'Helsinki-NLP/opus-mt-{source_lang}-{target_lang}'
                    try:
                        self.translation_models[f"{source_lang}-{target_lang}"] = MarianMTModel.from_pretrained(model_name)
                        self.tokenizers[f"{source_lang}-{target_lang}"] = MarianTokenizer.from_pretrained(model_name)
                    except Exception as e:
                        logger.warning(f"Could not load model {model_name}: {e}")

    def translate_text(self, text: str, source_lang: str, target_lang: str) -> str:
        """Translate text with caching"""
        if source_lang == target_lang:
            return text

        cache_key = f"{text}:{source_lang}:{target_lang}"
        if cache_key in self.translation_cache:
            return self.translation_cache[cache_key]

        try:
            # Use English as intermediate language for unsupported direct translations
            if f"{source_lang}-{target_lang}" not in self.translation_models:
                if source_lang != 'en':
                    text = self.translate_text(text, source_lang, 'en')
                if target_lang != 'en':
                    text = self.translate_text(text, 'en', target_lang)
                return text

            model = self.translation_models[f"{source_lang}-{target_lang}"]
            tokenizer = self.tokenizers[f"{source_lang}-{target_lang}"]

            inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True)
            translated = model.generate(**inputs)
            translated_text = tokenizer.decode(translated[0], skip_special_tokens=True)

            self.translation_cache[cache_key] = translated_text
            return translated_text

        except Exception as e:
            logger.error(f"Translation error: {e}")
            return text

    def detect_language(self, text: str) -> str:
        """Detect language with fallback options"""
        try:
            detected = detect(text)
            return detected if detected in self.language_pairs else 'en'
        except:
            return 'en'

    def analyze_sentiment(self, text: str, language: str) -> Dict:
        """Analyze sentiment of text"""
        if language != 'en':
            text = self.translate_text(text, language, 'en')
        return self.sentiment_analyzer(text)[0]

    def extract_keywords(self, text: str, language: str) -> Set[str]:
        """Extract important keywords from text"""
        try:
            # Translate to English if needed
            if language != 'en':
                text = self.translate_text(text, language, 'en')

            # Tokenize and remove stopwords
            stop_words = set(stopwords.words('english'))
            tokens = word_tokenize(text.lower())
            keywords = {word for word in tokens if word.isalnum() and word not in stop_words}

            return keywords
        except Exception as e:
            logger.error(f"Keyword extraction error: {e}")
            return set()

    def add_document(self,
                    text: str,
                    language: str,
                    category: str = None,
                    metadata: Dict = None,
                    doc_id: str = None):
        """Add document with enhanced metadata"""
        if not doc_id:
            doc_id = f"doc_{len(self.documents)}"

        # Extract keywords and analyze sentiment
        keywords = self.extract_keywords(text, language)
        sentiment = self.analyze_sentiment(text, language)

        # Create document with enhanced metadata
        doc = Document(
            id=doc_id,
            text=text,
            language=language,
            category=category,
            keywords=keywords,
            metadata={
                **(metadata or {}),
                'sentiment': sentiment,
                'added_date': datetime.datetime.now()
            }
        )

        # Store document and update indexes
        self.documents[doc_id] = doc
        self.language_indexes[language].append(doc_id)
        if category:
            self.category_indexes[category].append(doc_id)
        for keyword in keywords:
            self.keyword_indexes[keyword].append(doc_id)

        # Update vectorizer
        if language not in self.vectorizers:
            self.vectorizers[language] = TfidfVectorizer()

        return doc_id

    def search_and_translate(self,
                           query: str,
                           source_lang: str = None,
                           target_lang: str = None,
                           top_k: int = 3,
                           category: str = None,
                           date_range: tuple = None,
                           sentiment: str = None) -> List[Dict]:
        """Enhanced search with filters and translations"""
        if not source_lang:
            source_lang = self.detect_language(query)
        if not target_lang:
            target_lang = 'en'

        results = []
        for doc_lang in set(doc.language for doc in self.documents.values()):
            translated_query = self.translate_text(query, source_lang, doc_lang)
            lang_results = self._search_in_language(translated_query, doc_lang, top_k)
            results.extend(lang_results)

        # Apply filters
        if category:
            results = [r for r in results if self.documents[r['id']].category == category]
        if date_range:
            start, end = date_range
            results = [r for r in results if start <= self.documents[r['id']].metadata['added_date'] <= end]
        if sentiment:
            results = [r for r in results if self.documents[r['id']].metadata['sentiment']['label'] == sentiment]

        # Sort and limit results
        results.sort(key=lambda x: x['score'], reverse=True)
        results = results[:top_k]

        # Translate results
        for result in results:
            result['original_text'] = result['text']
            result['text'] = self.translate_text(result['text'], result['language'], target_lang)
            result['translated_to'] = target_lang
            result['sentiment'] = self.documents[result['id']].metadata['sentiment']
            result['keywords'] = list(self.documents[result['id']].keywords)

        return results

    def _search_in_language(self, query: str, language: str, top_k: int) -> List[Dict]:
        """Enhanced language-specific search"""
        if language not in self.vectorizers:
            return []

        docs = [doc for doc in self.documents.values() if doc.language == language]
        if not docs:
            return []

        vectorizer = self.vectorizers[language]
        doc_vectors = vectorizer.fit_transform([doc.text for doc in docs])
        query_vector = vectorizer.transform([query])

        similarities = cosine_similarity(query_vector, doc_vectors).flatten()
        top_indices = similarities.argsort()[-top_k:][::-1]

        results = []
        for idx in top_indices:
            doc = docs[idx]
            results.append({
                'id': doc.id,
                'text': doc.text,
                'language': doc.language,
                'score': float(similarities[idx]),
                'category': doc.category,
                'metadata': doc.metadata
            })

        return results

def run_interactive_system():
    """Run the enhanced interactive system"""
    retriever = EnhancedCrossLanguageRetriever()

    # Add comprehensive sample documents
    sample_docs = [
        # Technology documents
        ("Artificial Intelligence and Machine Learning are revolutionizing technology and business processes worldwide.",
         "en", "Technology"),
        ("El aprendizaje automático y la inteligencia artificial están transformando los procesos empresariales.",
         "es", "Technology"),
        ("L'intelligence artificielle transforme la façon dont nous travaillons et vivons.",
         "fr", "Technology"),

        # Business documents
        ("Cloud computing enables scalable and efficient business solutions for modern enterprises.",
         "en", "Business"),
        ("Die Cloud-Computing-Technologie ermöglicht skalierbare Geschäftslösungen.",
         "de", "Business"),

        # Science documents
        ("Recent breakthroughs in quantum computing show promising results for future applications.",
         "en", "Science"),
        ("La computación cuántica muestra resultados prometedores en investigaciones recientes.",
         "es", "Science"),

        # Healthcare documents
        ("Digital healthcare solutions are improving patient care and medical research.",
         "en", "Healthcare"),
        ("Le soluzioni sanitarie digitali stanno migliorando l'assistenza ai pazienti.",
         "it", "Healthcare"),

        # Entertainment documents
        ("Virtual reality technology is transforming the gaming and entertainment industry.",
         "en", "Entertainment"),
        ("La réalité virtuelle transforme l'industrie du jeu et du divertissement.",
         "fr", "Entertainment")
    ]

    # Add documents with metadata
    for text, lang, category in sample_docs:
        retriever.add_document(
            text=text,
            language=lang,
            category=category,
            metadata={
                'source': 'sample_data',
                'timestamp': datetime.datetime.now()
            }
        )

    print("\nWelcome to Enhanced Cross-Language Information Retrieval System")
    print("Available languages:", ", ".join(f"{code}: {name}" for code, name in retriever.language_pairs.items()))
    print("Available categories: Technology, Business, Science, Healthcare, Entertainment")

    while True:
        try:
            # Get user input
            query = input("\nEnter your search query (or 'quit' to exit): ")
            if query.lower() == 'quit':
                break

            source_lang = input("Enter your query language [auto-detect]: ").strip() or None
            target_lang = input("Enter desired result language [en]: ").strip() or 'en'
            category = input("Enter category filter (optional): ").strip() or None

            # Perform search
            results = retriever.search_and_translate(
                query=query,
                source_lang=source_lang,
                target_lang=target_lang,
                category=category,
                top_k=5
            )

            # Display enhanced results
            print("\nSearch Results:")
            for i, result in enumerate(results, 1):
                print(f"\nResult {i}:")
                print(f"Original ({result['language']}): {result['original_text']}")
                print(f"Translated ({result['translated_to']}): {result['text']}")
                print(f"Category: {result['category']}")
                print(f"Sentiment: {result['sentiment']['label']} ({result['sentiment']['score']:.3f})")
                print(f"Keywords: {', '.join(result['keywords'])}")
                print(f"Relevance Score: {result['score']:.3f}")

        except Exception as e:
            print(f"An error occurred: {e}")
            print("Please try again.")

if __name__ == "__main__":
    run_interactive_system()

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

...


Welcome to Enhanced Cross-Language Information Retrieval System
Available languages: en: English, es: Spanish, fr: French, de: German, it: Italian, pt: Portuguese, nl: Dutch, pl: Polish, ru: Russian, zh: Chinese, ja: Japanese, ko: Korean, ar: Arabic, hi: Hindi
Available categories: Technology, Business, Science, Healthcare, Entertainment

Enter your search query (or 'quit' to exit): Artificial Intelligence
Enter your query language [auto-detect]: en
Enter desired result language [en]: hi
Enter category filter (optional): 


ERROR:__main__:Translation error: maximum recursion depth exceeded while calling a Python object
ERROR:__main__:Translation error: maximum recursion depth exceeded while calling a Python object
ERROR:__main__:Translation error: maximum recursion depth exceeded while calling a Python object
ERROR:__main__:Translation error: maximum recursion depth exceeded while calling a Python object
ERROR:__main__:Translation error: maximum recursion depth exceeded while calling a Python object
ERROR:__main__:Translation error: maximum recursion depth exceeded while calling a Python object
ERROR:__main__:Translation error: maximum recursion depth exceeded while calling a Python object
ERROR:__main__:Translation error: maximum recursion depth exceeded while calling a Python object
ERROR:__main__:Translation error: maximum recursion depth exceeded while calling a Python object
ERROR:__main__:Translation error: maximum recursion depth exceeded while calling a Python object
ERROR:__main__:Translation err


Search Results:

Result 1:
Original (fr): L'intelligence artificielle transforme la façon dont nous travaillons et vivons.
Translated (hi): Artificial intelligence transforms the way we work and live.
Category: Technology
Sentiment: POSITIVE (1.000)
Keywords: 
Relevance Score: 0.485

Result 2:
Original (en): Artificial Intelligence and Machine Learning are revolutionizing technology and business processes worldwide.
Translated (hi): Artificial Intelligence and Machine Learning are revolutionizing technology and business processes worldwide.
Category: Technology
Sentiment: POSITIVE (0.999)
Keywords: 
Relevance Score: 0.442

Result 3:
Original (es): El aprendizaje automático y la inteligencia artificial están transformando los procesos empresariales.
Translated (hi): Machine learning and artificial intelligence are transforming business processes.
Category: Technology
Sentiment: POSITIVE (0.999)
Keywords: 
Relevance Score: 0.436

Result 4:
Original (it): Le soluzioni sanitarie digitali 

ERROR:__main__:Translation error: maximum recursion depth exceeded while calling a Python object
ERROR:__main__:Translation error: maximum recursion depth exceeded while calling a Python object
ERROR:__main__:Translation error: maximum recursion depth exceeded while calling a Python object
ERROR:__main__:Translation error: maximum recursion depth exceeded while calling a Python object
ERROR:__main__:Translation error: maximum recursion depth exceeded while calling a Python object
ERROR:__main__:Translation error: maximum recursion depth exceeded while calling a Python object
ERROR:__main__:Translation error: maximum recursion depth exceeded while calling a Python object
ERROR:__main__:Translation error: maximum recursion depth exceeded while calling a Python object
ERROR:__main__:Translation error: maximum recursion depth exceeded while calling a Python object
ERROR:__main__:Translation error: maximum recursion depth exceeded while calling a Python object
ERROR:__main__:Translation err


Search Results:

Result 1:
Original (it): Le soluzioni sanitarie digitali stanno migliorando l'assistenza ai pazienti.
Translated (en): Digital healthcare solutions are improving patient care.
Category: Healthcare
Sentiment: POSITIVE (0.997)
Keywords: 
Relevance Score: 0.000

Result 2:
Original (de): Die Cloud-Computing-Technologie ermöglicht skalierbare Geschäftslösungen.
Translated (en): Cloud computing technology enables scalable business solutions.
Category: Business
Sentiment: POSITIVE (0.999)
Keywords: 
Relevance Score: 0.000

Result 3:
Original (en): Virtual reality technology is transforming the gaming and entertainment industry.
Translated (en): Virtual reality technology is transforming the gaming and entertainment industry.
Category: Entertainment
Sentiment: POSITIVE (1.000)
Keywords: 
Relevance Score: 0.000

Result 4:
Original (en): Digital healthcare solutions are improving patient care and medical research.
Translated (en): Digital healthcare solutions are improving pati